# Par versus indexed coupons

(Based on [a question asked by KK](https://sourceforge.net/p/quantlib/mailman/message/32902476/) on the QuantLib mailing list. Thanks!)

In [1]:
from QuantLib import *
import pandas as pd

today = Date(7,January,2013)
Settings.instance().evaluationDate = today

#### The statement of the case

User KK was pricing an interest-rate swap.  In the interest of brevity, I'll skip the part where he bootstrapped a LIBOR curve (there are other notebooks showing that in detail) and instantiate the resulting curve directly from the resulting forward rates.

In [2]:
dates, forwards = zip(*[(Date(7,1,2013), 0.03613672438543303),
                        (Date(8,4,2013), 0.03613672438543303),
                        (Date(8,7,2013), 0.033849133719219514),
                        (Date(7,1,2014), 0.03573931373272106),
                        (Date(7,7,2014), 0.03445303757052511)])
libor_curve = ForwardCurve(dates, forwards, Actual365Fixed())

Here is the floating leg of the swap; we don't need to care about the fixed leg.

In [3]:
index = GBPLibor(Period(6,Months),
                 YieldTermStructureHandle(libor_curve))

calendar = index.fixingCalendar()
nominal = 1000000
length = 1
maturity = calendar.advance(today,length,Years)
adjustment = index.businessDayConvention()

schedule = Schedule(today, maturity,
                    index.tenor(), calendar,
                    adjustment, adjustment,
                    DateGeneration.Backward, False)

floating_leg = IborLeg([nominal], schedule,
                       index, index.dayCounter()) 

Next, KK set out to do some cash-flow analysis. He reproduced the coupon amounts by multiplying the LIBOR fixing, the notional, and the accrual period; the actual code was different, but the calculations are the same I'm doing here:

In [4]:
df = pd.DataFrame()

dates = list(schedule)
df['fixing date'] = dates[:-1]
df['index fixing'] = [ index.fixing(d) for d in df['fixing date'] ]
df['start date'] = dates[:-1]
df['end date'] = dates[1:]
df['days'] = df['end date'] - df['start date']
df['accrual period'] = df['days']/365

df['amount'] = df['index fixing'] * nominal * df['accrual period']

df

,fixing date,index fixing,start date,end date,days,accrual period,amount
0,"January 7th, 2013",0.035300,"January 7th, 2013","July 8th, 2013",182,0.49863,17601.6
1,"July 8th, 2013",0.036056,"July 8th, 2013","January 7th, 2014",183,0.50137,18077.4


Unfortunately, the results for the second coupon don't agree with what the library says:

In [5]:
df2 = pd.DataFrame({'amount': [ c.amount() for c in floating_leg ],
                    'rate': [ as_coupon(c).rate() for c in floating_leg ]})
df2

,amount,rate
0,17601.643836,0.035300
1,18080.116395,0.036061


The difference (in the rate, and thus the amount) is small but well above the expected precision for the calculations.

#### Where's the problem?

Let's go through the calculations again.  The second coupon fixes on the expected date, and the forecast for the fixing is the same KK obtained.

In [6]:
coupon = as_floating_rate_coupon(floating_leg[1])

print(coupon.fixingDate())
print(index.fixing(coupon.fixingDate()))

July 8th, 2013
0.036056087457623655


The fixing is also consistent with what we can forecast from the LIBOR curve, given the start and end date of the underlying tenor:

In [7]:
startDate = index.valueDate(coupon.fixingDate())
endDate = index.maturityDate(startDate)
print(startDate)
print(endDate)

July 8th, 2013
January 8th, 2014


In [8]:
print(libor_curve.forwardRate(startDate, endDate,
                              coupon.dayCounter(), Simple))

3.605609 % Actual/365 (Fixed) simple compounding


The above is, in fact, the calculation performed in the `index.fixing` method.

Why does the coupon return a different rate, then?

The problem is that, for historical reasons, the coupon is calculated at par; that is, the floating rate is calculated over the duration of the coupon. Due to the constraints of the schedule, the end of the coupon doesn't correspond to the end of the LIBOR tenor...

In [9]:
couponStart = coupon.accrualStartDate()
couponEnd = coupon.accrualEndDate()
print(couponStart)
print(couponEnd)

July 8th, 2013
January 7th, 2014


...and therefore, the calculated rate is different:

In [10]:
print(libor_curve.forwardRate(couponStart, couponEnd,
                              coupon.dayCounter(), Simple))

3.606143 % Actual/365 (Fixed) simple compounding


The coupon amount is consistent with the rate above...

In [11]:
coupon.rate()

0.0360614343399347

...and so is the amount:

In [12]:
coupon.rate() * nominal * coupon.accrualPeriod()

18080.116395090554

In [13]:
coupon.amount()

18080.11639509055

Was it a good idea to use par coupons?  Hard to say.  They are used in textbook examples, which one might want to reproduce.

In any case, I've heard arguments against both calculations.  The one against using the forecast index fixing goes that the rate would be accrued over a period which is different from the one over which it was calculated, and thus it will require a small convexity adjustment.  Personally, the argument failed to persuade me, since with par coupons we're using the wrong rate over the right period and therefore we're introducing an error anyway; I doubt that it's smaller than the missing convexity adjustment.  Moreover, the value of the swap is going to jump as soon as the coupon rate is actually fixed, forcing an adjustment of the P&L.

The good news is that you can choose which one to use: there's a configuration flag for the library that allows you to use the forecast of the fixing.  The bad news is that this requires the recompilation of both the C++ library and the Python module.  It would be better if the choice could be made at run-time; I describe the details in [_Implementing QuantLib_](https://leanpub.com/implementingquantlib/), including a few glitches that this might cause.  For the time being, you'll have to make do.